In [3]:
import requests
import pandas as pd
import time

def fetch_batch_compound_data(batch_cids):
    base_url = "https://rest.kegg.jp/get/"
    query = '+'.join(batch_cids)
    response = requests.get(f"{base_url}{query}")
    if response.status_code == 200:
        return response.text  # Return the raw response for further processing
    else:
        raise Exception(f"Failed to fetch data for batch: {batch_cids}")

def parse_batch_response(batch_response):
    # Initialize an empty dictionary to store CID to formula mappings
    cid_to_formula = {}

    # Split the batch response by the '///' delimiter which separates entries
    entries = batch_response.strip().split('///\n')

    for entry in entries:
        # Split the entry into lines and filter out empty lines
        lines = [line.strip() for line in entry.split('\n') if line.strip()]
        
        # Initialize variables to store the current CID and formula
        current_cid = None
        current_formula = None

        for line in lines:
            # Check if the line contains a CID
            if line.startswith('ENTRY'):
                # Extract the CID from the line
                current_cid = line.split()[1]

            # Check if the line contains a formula
            elif line.startswith('FORMULA'):
                # Extract the formula from the line
                current_formula = line.split(maxsplit=1)[1].strip()

        # If both CID and formula were found, add them to the dictionary
        if current_cid and current_formula:
            cid_to_formula[current_cid] = current_formula

    return cid_to_formula

def get_kegg_compound_formulas(batch_cids, batch_size=10):
    cid_to_formula = {}
    for i in range(0, len(batch_cids), batch_size):
        batch = batch_cids[i:i + batch_size]
        batch_response = fetch_batch_compound_data(batch)
        batch_data = parse_batch_response(batch_response)
        cid_to_formula.update(batch_data)
    return cid_to_formula

# Example usage
df = pd.read_excel('TableS2_BioticRxns.xlsx') # Load your DataFrame

# Extract unique CIDs from Reactants and Products columns
unique_cids = set(cid for sublist in df['Reactants'].str.cat(df['Products'], sep=',').str.split(',') for cid in sublist if cid.startswith('C'))

# Get formulas in batches
cid_to_formula = get_kegg_compound_formulas(list(unique_cids))

# Replace CIDs with formulas in Reactants and Products
df['Reactants'] = df['Reactants'].str.split(',').apply(lambda x: ','.join(cid_to_formula.get(cid, cid) for cid in x))
df['Products'] = df['Products'].str.split(',').apply(lambda x: ','.join(cid_to_formula.get(cid, cid) for cid in x))

# Show the modified DataFrame
print(df)


      Serial Reaction_entry                       Reactants Reactant_stoi  \
0          1       R06974.a  CH2O2,C10H16N5O13P3,C7H15N2O8P         1,1,1   
1          2       R06974.b  C10H15N5O10P2,H3PO4,C8H15N2O9P         1,1,1   
2          3       RX0001.a                             H2O             1   
3          4       RX0001.b                            H,OH           1,1   
4          5       RX0002.a                         H2O,NH3           1,1   
...      ...            ...                             ...           ...   
8397    8398       R11937.b     C10H15N5O11P2,C69H115NO17P2           1,1   
8398    8399       R11938.a     C16H25N5O16P2,C69H115NO17P2           2,1   
8399    8400       R11938.b     C10H15N5O11P2,C81H135NO27P2           2,1   
8400    8401       R11941.a     C10H16N5O13P3,C5H10N2O3,H2O         1,1,1   
8401    8402       R11941.b    C10H14N5O7P,H3PO4,C5H11N2O6P         1,1,1   

                            Products Product_stoi  \
0     C10H15N5O10P2,H3

In [10]:
df.to_csv('TableS2_bioticrxns', index=False)

In [8]:
df_2 = pd.read_excel('TableS1_AbioticRxns.xlsx')
df_2.to_csv('TableS1_Abioticrxns', index=False)